Before you get into this I just wanna say I'm nowhere near where I wanted to be for this draft, I've been working on this for the past 6 hours straight and still barely have anything working. I've managed to pull and tidy all the data from a singular match (towards the bottom somewhere) but I'm having trouble creating functions to automate this process so I can expand the dataset beyond a single match. I think my plans for this project far exceeded my capabilities, so my final project will definitely have to be scaled back. If I had better knowledge of pandas or python in general maybe I could have met my original expectations but it is what it is. 

I love the game PUBG, it has a very nostalgic place in my heart (despite being only 6 years old) and I have a lot of fond memories with friends on the game. I wanted to see what statistically makes a player place higher in the match. Specific weapons, playstyles, landing location, etc.

Kind of just doing my own thing, I'm not (consciously) following in the footsteps of anyone else

The source of the data is the PUBG API, collected by the game as people play it. It could be valuable to gain insights on the game that you otherwise couldn't.

The data includes players, player stats, match history, leaderboards, specific matches, telemetry events within matches, etc. I'm going to use anything I believe would be interesting to observe in relation to how a player performs in a match.

I managed to bring together and tidy data for a specific match like player name, placement, kills, distance travelled, death location, carepackages looted, time survived, and damage dealt. I have yet to analyze anything because I still don't have a big enough data set.

N/A

N/A

Oh also now I'm getting 404s on stuff that was previously working and I didn't think I changed anything but they were just for testing at the beginning so I don't think its a huge deal but was working on one point haha.

In [511]:
#Explain the purpose of the analysis (what questions and motivations drive you?)
#Explain the starting point (what brought you here, are there others you are following?)
#Explain the source of the data (where did this data come from, what value does it have?)
#Explain the structure of the data (how is it organized, what pieces of information are included, which pieces will you use?) (these last two are the data ethnographies portions)
#Explain what you did (how was the data cleaned/changed, what analysis was conducted and why?)
#Explain what the results mean (what have you learned, what new questions need to be asked, what are your limitations/blindspots?)
#Explain your conclusion (what should your audience take away from the report?)

In [15]:
import pandas as pd
import json
import requests
import base64
import urllib

reading in the api key

In [16]:
app_key = pd.read_csv("keys/PUBG.txt")

Creating api call function and header

In [17]:
def api_call(endpoint_url, api_header):
    response = requests.get(endpoint_url, headers = api_header)
    print(response.status_code)
    return response.json()

In [18]:
header = {'Authorization':'Bearer {}'.format(app_key['Key'].iloc[0]), 'Accept':'application/vnd.api+json'}

Testing the call

In [19]:
test = api_call("https://api.pubg.com/shards/steam/players?filter[playerNames]=ChangChunYuanMan", header)
test_df = pd.DataFrame(test['data'][0])
test_df

200


,type,id,attributes,relationships,links
name,player,account.619b02706fc04dcea87889fb2e621f42,ChangChunYuanMan,NaN,NaN
stats,player,account.619b02706fc04dcea87889fb2e621f42,None,NaN,NaN
titleId,player,account.619b02706fc04dcea87889fb2e621f42,pubg,NaN,NaN
shardId,player,account.619b02706fc04dcea87889fb2e621f42,steam,NaN,NaN
patchVersion,player,account.619b02706fc04dcea87889fb2e621f42,,NaN,NaN
banType,player,account.619b02706fc04dcea87889fb2e621f42,Innocent,NaN,NaN
clanId,player,account.619b02706fc04dcea87889fb2e621f42,clan.b02c3427046d433d86fd9f0f4924b0da,NaN,NaN
assets,player,account.619b02706fc04dcea87889fb2e621f42,NaN,{'data': []},NaN
matches,player,account.619b02706fc04dcea87889fb2e621f42,NaN,"{'data': [{'type': 'match', 'id': '5529286a-ac...",NaN
self,player,account.619b02706fc04dcea87889fb2e621f42,NaN,NaN,https://api.pubg.com/shards/steam/players/acco...


Getting data for this specific match

In [500]:
thematch = api_call("https://api.pubg.com/shards/steam/matches/96cb6571-a20a-11ef-ad2c-0ea585c69622", header)

404


Getting player stats

In [502]:
matchdf = pd.DataFrame(thematch['included'])
stats_df = pd.DataFrame(matchdf['attributes'][0])
stats_df

KeyError: 'included'

Trying to get telemetry data, starting with the id for it

In [ ]:
thematch['data']['relationships']['assets']

Getting the index for the address

In [ ]:
matchdf[matchdf['id'] == '96cb6571-a20a-11ef-ad2c-0ea585c69622']['attributes']

Creating a dataframe out of the telemetry data (Thanks for the help here your solution was pretty much dead on)

In [510]:
URL = matchdf[matchdf['id'] == '96cb6571-a20a-11ef-ad2c-0ea585c69622']['attributes'][68]['URL']
telemetry = pd.read_json(URL)
tele_df = pd.DataFrame(telemetry)

NameError: name 'matchdf' is not defined

_D is the Time <br>
_T is the type of event <br>
common is the status of the current match, isGame = 0 is before match start, 0.1 is on the plane, 0.5 is before zone, 1 is the first zone appears, 1.5 is when it shrinks, 2 is when the next one starts, and so on <br>
character is info about the player the event refers to <br> mostly everything else is specific to the event type

In [ ]:
tele_df

I wanted to figure out how to get a list of all the specific types of event. I had a lot of trouble figuring out how to locate the index of a row based on its value, but stack overflow came in clutch as usual

In [ ]:
tele_df['_T'].index[tele_df['_T']=='LogParachuteLanding'].tolist()

Here I can get the player behind the event and where it happened

In [ ]:
tele_df['_D'][2156]
tele_df.iloc[2156]['character']

I need to get the season id for the leaderboard I wanna get

In [22]:
api_call("https://api.pubg.com/shards/steam/seasons/", header)

200


{'data': [{'type': 'season',
   'id': 'division.bro.official.pc-2018-33',
   'attributes': {'isCurrentSeason': True, 'isOffseason': False}},
  {'type': 'season',
   'id': 'division.bro.official.pc-2018-32',
   'attributes': {'isCurrentSeason': False, 'isOffseason': False}},
  {'type': 'season',
   'id': 'division.bro.official.pc-2018-31',
   'attributes': {'isOffseason': False, 'isCurrentSeason': False}},
  {'type': 'season',
   'id': 'division.bro.official.pc-2018-30',
   'attributes': {'isCurrentSeason': False, 'isOffseason': False}},
  {'type': 'season',
   'id': 'division.bro.official.pc-2018-29',
   'attributes': {'isCurrentSeason': False, 'isOffseason': False}},
  {'type': 'season',
   'id': 'division.bro.official.pc-2018-28',
   'attributes': {'isCurrentSeason': False, 'isOffseason': False}},
  {'type': 'season',
   'id': 'division.bro.official.pc-2018-27',
   'attributes': {'isCurrentSeason': False, 'isOffseason': False}},
  {'type': 'season',
   'id': 'division.bro.official.pc

The most recent leaderboard that works is season 6??? It is currently season 33.

In [404]:
leaderboard = api_call("https://api.pubg.com/shards/pc-na/leaderboards/division.bro.official.pc-2018-06/solo", header)
leaderboard

200


{'data': {'type': 'leaderboard',
  'id': 'a14ee5e5-0a41-4d35-95dc-6159ba57e24a',
  'attributes': {'gameMode': 'solo',
   'seasonId': 'division.bro.official.pc-2018-06',
   'shardId': 'pc-na'},
  'relationships': {'players': {'data': [{'type': 'player',
      'id': 'account.2f6161f9becd4f8d9bcac25c5f049be8'},
     {'type': 'player', 'id': 'account.a2465801e62143b3b1fca6cd29949889'},
     {'type': 'player', 'id': 'account.ae876e3afcd243b4b6f3b96c6d306d92'},
     {'type': 'player', 'id': 'account.a33b5918828d48dc93d68f6b3429e9e9'},
     {'type': 'player', 'id': 'account.52aa8c14e05c45eea359fe62c619fc71'},
     {'type': 'player', 'id': 'account.bce91fed038f4b408584473904996a9e'},
     {'type': 'player', 'id': 'account.ce8e3288f4b645528e75a938e485f0de'},
     {'type': 'player', 'id': 'account.fe8dba6b2f7049289ea75f030bd48a3b'},
     {'type': 'player', 'id': 'account.336af7f6f0344dbc8c35516a74cbed14'},
     {'type': 'player', 'id': 'account.b5b3a8c576b5426d985090ef1b0c4966'},
     {'type': '

In [352]:
leaderboard.keys()
#leaderboard['data']['relationships']['players']['data']
leader_df = pd.DataFrame(leaderboard['data']['relationships']['players']['data'])
leader_df

,type,id
0,player,account.2f6161f9becd4f8d9bcac25c5f049be8
1,player,account.a2465801e62143b3b1fca6cd29949889
2,player,account.ae876e3afcd243b4b6f3b96c6d306d92
3,player,account.a33b5918828d48dc93d68f6b3429e9e9
4,player,account.52aa8c14e05c45eea359fe62c619fc71
5,player,account.bce91fed038f4b408584473904996a9e
6,player,account.ce8e3288f4b645528e75a938e485f0de
7,player,account.fe8dba6b2f7049289ea75f030bd48a3b
8,player,account.336af7f6f0344dbc8c35516a74cbed14
9,player,account.b5b3a8c576b5426d985090ef1b0c4966


Here I can see that I can get recent matches from a player

In [420]:
player = api_call("https://api.pubg.com/shards/steam/players/"+leader_df['id'][15], header)
player

200


{'data': {'type': 'player',
  'id': 'account.45c7abec405844ec9f577f38cf8a3c06',
  'attributes': {'stats': None,
   'titleId': 'pubg',
   'shardId': 'steam',
   'patchVersion': '',
   'banType': 'Innocent',
   'clanId': '',
   'name': 'AIIEN6666666'},
  'relationships': {'assets': {'data': []},
   'matches': {'data': [{'type': 'match',
      'id': 'e1a9007e-e516-4e7e-9279-5405fbaa0169'},
     {'type': 'match', 'id': '02f6ab64-ca10-4e34-9cc7-866cd6fbb810'},
     {'type': 'match', 'id': '801d1445-28de-4077-919d-d0c9588d2e45'},
     {'type': 'match', 'id': '4173f104-c89b-4dd4-aa27-43c9b7bb01a3'},
     {'type': 'match', 'id': 'a5fd33ed-9b38-4d3d-8c58-19e17260adb3'},
     {'type': 'match', 'id': '801f67d4-d08b-46a0-b1ad-e5299eba55d4'},
     {'type': 'match', 'id': 'dd0b0708-a28e-4015-ba3b-8a7df9a5cb7a'},
     {'type': 'match', 'id': '24b2af19-1483-46ea-9ebe-947dc4678e65'},
     {'type': 'match', 'id': 'fa6b715c-3059-4bbb-a50e-65a4ee21566e'}]}},
  'links': {'self': 'https://api.pubg.com/shard

This was automation stuff that didn't work because the way I was trying to get recent player matches ended up throwing an error if a player didn't have any recent matches and I don't know how to fix that edge case.

In [400]:
#def getRecentMatches(player_id):
  #  match_collection = []
  #  player = api_call("https://api.pubg.com/shards/steam/players/"+player_id, header)
   # for x in range(1):
   #     match_collection.append(player['data']['relationships']['matches']['data'][x]['id'])
   # return match_collection

In [456]:
#match_ids = []
#for x in range(20):
  #  match_ids.append(getRecentMatches(leader_df['id'][x]))
#match_ids.append(getRecentMatches(leader_df['id'][2]))
#leader_df['id'][2]

In [514]:
match_collection

['3b84b9dd-6383-4162-94b0-b96c54c403b4',
 'b675b5f3-9f99-443e-959c-4e016ff0f050',
 'bf958047-2c8a-4185-b3f1-7e728c61f699',
 '9544b04d-4038-4578-9306-29649c578e02',
 'c58a44d5-1122-4fce-a3a0-529401a858c7',
 'fccf2ef1-13c0-4d67-bdaa-97eee812d5a6',
 '7477a20c-14c2-45f7-87f5-65a880bd2f17',
 'a4cc422d-2a18-4528-918b-04a26675f1aa',
 '96de0dd6-0286-45aa-a51f-b3f6625e7cbf',
 '74a8119f-c7f3-4266-9b88-34ad768aac7a',
 '2d60d3d9-c8ab-45a4-b16d-04e72cb3bc6f',
 '47f08dcb-2ce3-4493-b6ec-12cc8266fdae',
 '2cd6fc40-40a2-4f62-8fa5-9d9be6b4fde9',
 '5ea35b02-da9f-4106-9608-5c5b711630fb',
 'e73e18e2-aa3e-40ce-b49e-f095e7e7f745',
 '211d0934-aa3a-4ae8-ba3d-6f4242a55528',
 'b0a9db41-a170-4bd1-8ec4-1618628e5a31',
 'af7da6d2-3104-4057-8548-511c120786b6',
 '4262cb27-0506-490f-b62a-523c152eb174',
 '1aa9b774-a952-46de-a9a3-d7b47bb0a38a',
 'b1256999-9075-4d86-86ed-68c9e47c551a',
 '4ee7bacd-c402-488f-87b3-4a4704865919',
 'f674c9ba-5146-48a2-b0dc-0447053606a3',
 '3cf1ba1f-a41d-4037-85df-e07fe810fa5b',
 '0735bda5-174e-

In [515]:
playerids = pd.DataFrame(match['data']['relationships']['rosters']['data'])
players = pd.DataFrame(playerids['id'])

In [30]:
idofmatch = match['data']['relationships']['assets']['data'][0]['id']

In [189]:
match_df = pd.DataFrame(match['included'])
indexx = match_df[match_df['id'] == idofmatch]['attributes'].index[0]
indexx


[{'type': 'participant',
  'id': 'c0bbcb5f-7f8c-4d2f-b4bd-7e924ef68a32',
  'attributes': {'shardId': 'steam',
   'stats': {'DBNOs': 0,
    'assists': 0,
    'boosts': 0,
    'damageDealt': 100,
    'deathType': 'byplayer',
    'headshotKills': 0,
    'heals': 4,
    'killPlace': 29,
    'killStreaks': 1,
    'kills': 1,
    'longestKill': 13.834639,
    'name': '666courier66',
    'playerId': 'account.bdbd6f7356b14b8b9926c0453751950c',
    'revives': 0,
    'rideDistance': 0,
    'roadKills': 0,
    'swimDistance': 0,
    'teamKills': 0,
    'timeSurvived': 408,
    'vehicleDestroys': 0,
    'walkDistance': 311.48767,
    'weaponsAcquired': 7,
    'winPlace': 38},
   'actor': ''}},
 {'type': 'roster',
  'id': 'ecdd61ad-262b-48c9-ac18-ddb6f43ae640',
  'attributes': {'stats': {'rank': 18, 'teamId': 100038},
   'won': 'false',
   'shardId': 'steam'},
  'relationships': {'team': {'data': None},
   'participants': {'data': [{'type': 'participant',
      'id': '7a284eb7-1ddc-4e42-b7c0-cc902b

figuring out how to find telemetry data

In [32]:
URLfound = match_df[match_df['id'] == idofmatch]['attributes'][indexx]['URL']
tele = pd.read_json(URLfound)
tele_dff = pd.DataFrame(tele)

In [60]:
deaths = tele_dff['_T'].index[tele_dff['_T']=='LogPlayerKillV2'].tolist()
#tele_dff.iloc[2426].keys()
deaths[1]
tele_dff.iloc[deaths[98]]['victim']

{'name': 'weqljyhzikfn',
 'teamId': 100061,
 'health': 0,
 'location': {'x': 169163.40625, 'y': 99687.09375, 'z': 772.4199829101562},
 'ranking': 2,
 'individualRanking': 2,
 'accountId': 'account.6487efe43bf245919349f4005815d39b',
 'isInBlueZone': False,
 'isInRedZone': False,
 'inSpecialZone': 'None',
 'isInVehicle': False,
 'zone': [],
 'type': 'user'}

This is the start of trying to automate this stuff with functions

In [437]:
def getmatch(matchid):
    match = api_call('https://api.pubg.com/shards/steam/matches/'+matchid,header)
    match_df = pd.DataFrame(match['included'])
    
    idofmatch = match['data']['relationships']['assets']['data'][0]['id']
    indexx = match_df[match_df['id'] == idofmatch]['attributes'].index[0]
    
    URLfound = match_df[match_df['id'] == idofmatch]['attributes'][indexx]['URL']
    tele = pd.read_json(URLfound)
    tele_df = pd.DataFrame(tele)
    return(match_df, tele_df)

In [506]:
def collect(tele_df):
    
    deaths = tele_df['_T'].index[tele_df['_T']=='LogPlayerKillV2'].tolist()
    
    rankings = pd.DataFrame()
    #tele_df.iloc[deaths[98]]['victim']
    for x in deaths:
        rankings = pd.concat([rankings, pd.DataFrame({
        "Position": [tele_df.iloc[x]['victim']['ranking']],
        "Player": [tele_df.iloc[x]['victim']['name']],
        "Type": [tele_df.iloc[x]['victim']['type']],
        "Death Location": [tele_df.iloc[x]['victim']['location']],
        "Death Reason": [tele_df.iloc[x]['killerDamageInfo']['damageReason']]
        })], ignore_index=True)
        

    matchend = tele_df['_T'].index[tele_df['_T']=='LogMatchEnd'][0]
    
    newranking = pd.DataFrame()
    
    info = tele_df.iloc[matchend]['characters']
    
    for y in range(0,100):
        newranking = pd.concat([newranking, pd.DataFrame({
        "Position": [info[x]['character']['ranking']],
        "Player": [info[x]['character']['name']],
        "Type": [info[x]['character']['type']]
        })], ignore_index=True)
    #tele_dff.iloc[25473]['characters'][1]['character']
    
    deaths_and_results = rankings.merge(newranking, how='outer')
    return deaths_and_results

And this is the end

In [488]:
rankings = pd.DataFrame()
tele_dff.iloc[deaths[98]]['victim']
for x in range(0,99):
    rankings = pd.concat([rankings, pd.DataFrame({
    "Position": [tele_dff.iloc[deaths[x]]['victim']['ranking']],
    "Player": [tele_dff.iloc[deaths[x]]['victim']['name']],
    "Type": [tele_dff.iloc[deaths[x]]['victim']['type']],
    "Death Location": [tele_dff.iloc[deaths[x]]['victim']['location']],
    "Death Reason": [tele_dff.iloc[deaths[x]]['killerDamageInfo']['damageReason']]
    
})], ignore_index=True)

#test = pd.DataFrame(tele_dff.iloc[deaths[98]]['victim'])
#test
#tele_dff.iloc[deaths[98]]['victim']


#row
for x in deaths:
    print(x)

3672
3923
3983
4295
4459
4683
4772
4896
5067
5161
5287
5583
5775
5827
5855
6059
6164
6290
6335
6670
6782
6951
6962
7286
7655
7944
8104
8127
8575
8753
8917
9141
9531
9587
9697
9784
10124
10382
10428
10862
10896
10954
11158
11168
11248
11707
11735
12024
12311
12325
12634
12676
12939
12984
13096
14267
14327
14864
15213
15283
15311
15396
15466
15503
15826
17270
17539
17632
18191
18396
19082
19250
19487
19803
20039
20097
20427
20804
20876
21054
21380
21990
22026
22094
22199
22491
22813
22954
23185
23265
24514
24578
24850
24915
24948
25051
25143
25226
25456


In [516]:
pd.set_option('display.max_rows', 1000)

matchend = tele_dff['_T'].index[tele_dff['_T']=='LogMatchEnd'][0]

newranking = pd.DataFrame()
info = tele_dff.iloc[matchend]['characters']
for x in range(0,100):
    newranking = pd.concat([newranking, pd.DataFrame({
    "Position": [info[x]['character']['ranking']],
    "Player": [info[x]['character']['name']],
    "Type": [info[x]['character']['type']]
    })], ignore_index=True)

newnew = pd.concat([rankings,newranking], ignore_index=True, )
newnewtest = rankings.merge(newranking, how='outer')
newnewtest


,Position,Player,Type,Death Location,Death Reason
0,1,Geazy1,user,NaN,NaN
1,2,weqljyhzikfn,user,"{'x': 169163.40625, 'y': 99687.09375, 'z': 772...",TorsoShot
2,3,Lecuma46,user,"{'x': 172854.921875, 'y': 103858.7734375, 'z':...",ArmShot
3,4,Bmakabak,user,"{'x': 163195.1875, 'y': 99802.140625, 'z': 927...",LegShot
4,5,gexh123,user,"{'x': 160866.609375, 'y': 106178.65625, 'z': 2...",ArmShot
5,6,Mooch_,user,"{'x': 188041.796875, 'y': 95341.203125, 'z': 3...",
6,7,lisaiyunb,user,"{'x': 182707.09375, 'y': 111069.046875, 'z': 1...",TorsoShot
7,8,my_love1851995,user,"{'x': 182630, 'y': 111026.3125, 'z': 1201.4599...",LegShot
8,9,0551-666666,user,"{'x': 229209.15625, 'y': 149814.484375, 'z': 1...",
9,10,Ganxin_,user,"{'x': 222304.34375, 'y': 176785.796875, 'z': 1...",


In [491]:
incl = pd.DataFrame(match['included'])
#incl['type']
participants = incl.loc[incl['type'] == 'participant'].index.tolist()
#match['included']
participants
newdf = pd.DataFrame()
for x in participants:
    #pd.concat([pd.DataFrame(incl['attributes'][x]),newdf], axis=0, ignore_index=True)
    #newdf.append(pd.DataFrame(incl['attributes'][x]))
    newdf = pd.concat([newdf, pd.DataFrame({
    "Player": [incl['attributes'][x]['stats']['name']],
    "Kills": [incl['attributes'][x]['stats']['kills']],
    "Dmg Dealt": [incl['attributes'][x]['stats']['damageDealt']],
    "Dist Walked": [incl['attributes'][x]['stats']['walkDistance']],
    "Longest Kill": [incl['attributes'][x]['stats']['longestKill']],
    "Time Survived": [incl['attributes'][x]['stats']['timeSurvived']],
    })], ignore_index=True)
#incl['attributes'][6]
#incl.keys()
#newdf
#pd.DataFrame(incl['attributes'][0])
newdf


,Player,Kills,Dmg Dealt,Dist Walked,Longest Kill,Time Survived
0,666courier66,1,100.000000,311.487670,13.834639,408
1,mutou1208,0,0.000000,137.145740,0.000000,102
2,Pershmi_ne,0,0.000000,603.157840,0.000000,559
3,Tarcz_a,0,0.000000,214.127080,0.000000,631
4,JKM_0728,0,0.000000,3.073987,0.000000,459
5,hld_100,0,0.000000,6.474634,0.000000,437
6,KR_CH_EESE,0,54.000000,7.355655,0.000000,417
7,FharlieW118,1,99.999990,202.967970,0.000000,521
8,Jik_uro,1,61.999996,0.687288,0.000000,419
9,Nisson10,0,0.000000,0.000000,0.000000,58


In [492]:
newestdf = newnewtest.merge(newdf,how='left')
newestdf

,Position,Player,Type,Death Location,Death Reason,Kills,Dmg Dealt,Dist Walked,Longest Kill,Time Survived
0,1,Geazy1,user,NaN,NaN,7,752.965200,3488.626500,103.041115,1386
1,2,weqljyhzikfn,user,"{'x': 169163.40625, 'y': 99687.09375, 'z': 772...",TorsoShot,5,488.154850,3179.946300,38.772892,1383
2,3,Lecuma46,user,"{'x': 172854.921875, 'y': 103858.7734375, 'z':...",ArmShot,1,13.775743,1776.148900,5.860226,1292
3,4,Bmakabak,user,"{'x': 163195.1875, 'y': 99802.140625, 'z': 927...",LegShot,4,545.650400,1897.220300,188.392880,1267
4,5,gexh123,user,"{'x': 160866.609375, 'y': 106178.65625, 'z': 2...",ArmShot,3,464.262500,1962.850000,94.144310,1237
5,6,Mooch_,user,"{'x': 188041.796875, 'y': 95341.203125, 'z': 3...",,1,135.133160,2836.782700,199.981750,1196
6,7,lisaiyunb,user,"{'x': 182707.09375, 'y': 111069.046875, 'z': 1...",TorsoShot,3,303.000000,2148.274200,41.474274,1192
7,8,my_love1851995,user,"{'x': 182630, 'y': 111026.3125, 'z': 1201.4599...",LegShot,0,259.409640,2348.848400,0.000000,1182
8,9,0551-666666,user,"{'x': 229209.15625, 'y': 149814.484375, 'z': 1...",,1,80.799920,3077.596000,66.445755,1100
9,10,Ganxin_,user,"{'x': 222304.34375, 'y': 176785.796875, 'z': 1...",,6,553.408500,889.916100,52.467613,1085


In [493]:
landing = tele_dff['_T'].index[tele_dff['_T']=='LogParachuteLanding'].tolist()
landing
landed_df = pd.DataFrame()
for x in landing:
    landed_df = pd.concat([landed_df, pd.DataFrame({
    "Player": [tele_dff['character'][x]['name']],
    "Landed Location": [tele_dff['character'][x]['location']]
    })], ignore_index=True)
#tele_dff['character'][3510]
landed_df

,Player,Landed Location
0,U-D1Ee,"{'x': 355427.375, 'y': 244170.671875, 'z': 119..."
1,o9Sy_-,"{'x': 343395.84375, 'y': 225850.5625, 'z': 668..."
2,Hau_Namutata,"{'x': 317907.5625, 'y': 242887.078125, 'z': 98..."
3,Nisson10,"{'x': 342992.71875, 'y': 227128.90625, 'z': 65..."
4,C7_Nickel,"{'x': 347023.84375, 'y': 222763.453125, 'z': 9..."
5,kaichuang-1,"{'x': 345963.28125, 'y': 219723.03125, 'z': 67..."
6,Jin__Bin,"{'x': 337364.34375, 'y': 261848.25, 'z': 1704...."
7,Jin__Bin,"{'x': 337352, 'y': 261846.734375, 'z': 1694.18..."
8,wus_e10,"{'x': 343790.3125, 'y': 289025.65625, 'z': 119..."
9,wus_e10,"{'x': 343789.90625, 'y': 289025.25, 'z': 1221...."


In [494]:

carepkg = tele_dff['_T'].index[tele_dff['_T']=='LogItemPickupFromCarepackage'].tolist()

carepkg_df = pd.DataFrame()
for x in carepkg:
    carepkg_df = pd.concat([carepkg_df, pd.DataFrame({
    "Player": [tele_dff['character'][x]['name']],
    "CarepackageLooted": ['True'],
    })], ignore_index=True, copy=False)
#tele_dff['character'][3510]
carepkg_df = carepkg_df.drop_duplicates()

In [495]:
final = newestdf.merge(carepkg_df,how='left')
final

,Position,Player,Type,Death Location,Death Reason,Kills,Dmg Dealt,Dist Walked,Longest Kill,Time Survived,CarepackageLooted
0,1,Geazy1,user,NaN,NaN,7,752.965200,3488.626500,103.041115,1386,True
1,2,weqljyhzikfn,user,"{'x': 169163.40625, 'y': 99687.09375, 'z': 772...",TorsoShot,5,488.154850,3179.946300,38.772892,1383,NaN
2,3,Lecuma46,user,"{'x': 172854.921875, 'y': 103858.7734375, 'z':...",ArmShot,1,13.775743,1776.148900,5.860226,1292,True
3,4,Bmakabak,user,"{'x': 163195.1875, 'y': 99802.140625, 'z': 927...",LegShot,4,545.650400,1897.220300,188.392880,1267,NaN
4,5,gexh123,user,"{'x': 160866.609375, 'y': 106178.65625, 'z': 2...",ArmShot,3,464.262500,1962.850000,94.144310,1237,NaN
5,6,Mooch_,user,"{'x': 188041.796875, 'y': 95341.203125, 'z': 3...",,1,135.133160,2836.782700,199.981750,1196,NaN
6,7,lisaiyunb,user,"{'x': 182707.09375, 'y': 111069.046875, 'z': 1...",TorsoShot,3,303.000000,2148.274200,41.474274,1192,NaN
7,8,my_love1851995,user,"{'x': 182630, 'y': 111026.3125, 'z': 1201.4599...",LegShot,0,259.409640,2348.848400,0.000000,1182,NaN
8,9,0551-666666,user,"{'x': 229209.15625, 'y': 149814.484375, 'z': 1...",,1,80.799920,3077.596000,66.445755,1100,NaN
9,10,Ganxin_,user,"{'x': 222304.34375, 'y': 176785.796875, 'z': 1...",,6,553.408500,889.916100,52.467613,1085,NaN


Basically my entire approach to this was make dataframe > make another and merge > make another and merge, and so on. I don't know if thats the best method but it works for the most part!

Manual match ids list I made because gathering them automatically from the leaderboard didn't work for reasons mentioned before.

In [422]:
match_collection = ['3b84b9dd-6383-4162-94b0-b96c54c403b4',
     'b675b5f3-9f99-443e-959c-4e016ff0f050',
     'bf958047-2c8a-4185-b3f1-7e728c61f699',
     '9544b04d-4038-4578-9306-29649c578e02',
     'c58a44d5-1122-4fce-a3a0-529401a858c7',
     'fccf2ef1-13c0-4d67-bdaa-97eee812d5a6',
     '7477a20c-14c2-45f7-87f5-65a880bd2f17',
     'a4cc422d-2a18-4528-918b-04a26675f1aa',
     '96de0dd6-0286-45aa-a51f-b3f6625e7cbf',
     '74a8119f-c7f3-4266-9b88-34ad768aac7a',
     '2d60d3d9-c8ab-45a4-b16d-04e72cb3bc6f',
     '47f08dcb-2ce3-4493-b6ec-12cc8266fdae',
     '2cd6fc40-40a2-4f62-8fa5-9d9be6b4fde9',
     '5ea35b02-da9f-4106-9608-5c5b711630fb',
     'e73e18e2-aa3e-40ce-b49e-f095e7e7f745',
     '211d0934-aa3a-4ae8-ba3d-6f4242a55528',
     'b0a9db41-a170-4bd1-8ec4-1618628e5a31',
     'af7da6d2-3104-4057-8548-511c120786b6',
     '4262cb27-0506-490f-b62a-523c152eb174',
     '1aa9b774-a952-46de-a9a3-d7b47bb0a38a',
     'b1256999-9075-4d86-86ed-68c9e47c551a',
     '4ee7bacd-c402-488f-87b3-4a4704865919',
     'f674c9ba-5146-48a2-b0dc-0447053606a3',
     '3cf1ba1f-a41d-4037-85df-e07fe810fa5b',
     '0735bda5-174e-43ee-9cd4-7ff98c6663bd',
     '4cdfc06a-0317-4bb0-b669-aa1d48f8f881',
     'e1ed2c9b-b9b7-4d7f-a2cd-4bf3b22fd8d9',
     '9f845b99-f9da-4817-8166-ba15940077e5',
     'ef68c7a9-10b1-4cd5-b548-87108d2d5a44',
     '7423f7f2-4acd-453e-b50e-0c88cd3cc357',
     '0b4adaba-fb20-40c8-8d0e-ffd805457b40',
     '78940f43-0b01-48ea-88a4-e83ef3f46d5d',
     '099175e5-2c19-4ad2-886c-91ecab47ef53',
     '1c4a25b2-432b-43c0-a6b1-1bf9449c36e5',
     'ba4eed5e-5602-4e6b-8e36-ba509bc163ae',
     'a8eb64da-01f1-46c0-8c24-aef9c4bebfa3',
     '1e28a132-5be4-4b4c-a1c0-ee2d6311b683',
     '286c91bf-1b87-4f3b-a445-04b0d46f56e9',
     'c629f704-347f-4108-b0bb-58cdd0abb6f1',
     'eaf82bb1-0a8a-434f-ad3c-003ffb4fafa7',
     '92d4c014-0726-452f-90a3-45078f47ced3',
     'e6da6a6d-092d-4210-a897-c35d9cf35f8b',
     '479d4c95-2516-49aa-98b2-3789279832ac',
     'b85228be-34ce-4624-9a43-a297a76b60da',
     'bce6d36e-8641-4a26-b416-2dc980d0f60d',
     '0b928924-0bf3-4b64-b1f8-9ccebf684c5b',
     '34a2b60a-0eb5-499c-afae-49735e479e18',
     'a28cda17-790d-4a6a-95bd-effdc89f9c9b',
     '502b5f14-89f0-4cbb-b280-f4b9e98d7842',
     'f63e2c97-559b-4069-9a34-de900b86225f',
     'b25f0fee-fba3-4b56-9d4b-374d5d48c64a',
     '8f4279ee-36b9-4334-8bac-c7c7c3bd89e9',
     '3d470152-3707-452a-893a-05d1cd3f3a25',
     '27989b7a-a373-4b7b-ac40-eba679aaaa2c',
     '5a66c05f-7ce0-45b2-b6f3-26a0f7e23775',
     'a3772cdf-f9fe-40ad-975a-dcf9f72b707d',
     '34c72043-27a7-4313-8ff9-0d442dd6e7dc',
     '02f6ab64-ca10-4e34-9cc7-866cd6fbb810',
     '801d1445-28de-4077-919d-d0c9588d2e45',
     '4173f104-c89b-4dd4-aa27-43c9b7bb01a3',
     'a5fd33ed-9b38-4d3d-8c58-19e17260adb3',
     '801f67d4-d08b-46a0-b1ad-e5299eba55d4',
     'dd0b0708-a28e-4015-ba3b-8a7df9a5cb7a',
     '24b2af19-1483-46ea-9ebe-947dc4678e65',
     'fa6b715c-3059-4bbb-a50e-65a4ee21566e']


In [509]:
final_df = pd.DataFrame()
#for x in range(len(match_collection)):
    #getmatch()
#print(getmatch(match_collection[0])[0])
collect(getmatch(match_collection[0])[1])

200


IndexError: list index out of range

I cannot for the life of me figure out why thats out of range :(